In [36]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [37]:
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [38]:
y_df = attrition_df[['Attrition', 'Department']]

In [39]:
X_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [40]:
X_df = attrition_df[X_columns]

In [41]:
X_df.dtypes

Age                         int64
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
JobInvolvement              int64
JobLevel                    int64
JobSatisfaction             int64
NumCompaniesWorked          int64
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
StockOptionLevel            int64
TotalWorkingYears           int64
TrainingTimesLastYear       int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
YearsWithCurrManager        int64
dtype: object

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Create a ColumnTransformer to handle numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', X_columns)
    ])

# Fit and transform the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Transform the testing data
X_test_preprocessed = preprocessor.transform(X_test)

In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_preprocessed)
X_test_scaled = scaler.transform(X_test_preprocessed)

In [45]:
enc_department = OneHotEncoder(sparse=False, handle_unknown='ignore')
enc_department.fit(y_train[['Department']])
y_train_encoded_department = enc_department.transform(y_train[['Department']])
y_test_encoded_department = enc_department.transform(y_test[['Department']])

/Users/katadhin1/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [46]:
enc_attrition = OneHotEncoder(sparse=False, handle_unknown='ignore')
enc_attrition.fit(y_train[['Attrition']])
y_train_encoded_attrition = enc_attrition.transform(y_train[['Attrition']])
y_test_encoded_attrition = enc_attrition.transform(y_test[['Attrition']])

/Users/katadhin1/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [47]:
input_shape = X_train_scaled.shape[1]

In [48]:
from tensorflow.keras.layers import Input

input_layer = Input(shape=(input_shape,))

In [49]:
from tensorflow.keras.layers import Dense

shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [50]:
department_branch = Dense(16, activation='relu')(shared_layer_2)
department_output = Dense(y_train_encoded_department.shape[1], activation='softmax')(department_branch)

In [51]:
from tensorflow.keras.layers import Dense

shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [52]:
department_branch = Dense(16, activation='relu')(shared_layer_2)
department_output = Dense(y_train_encoded_department.shape[1], activation='softmax')(department_branch)

In [53]:
attrition_branch = Dense(16, activation='relu')(shared_layer_2)
attrition_output = Dense(y_train_encoded_attrition.shape[1], activation='sigmoid')(attrition_branch)

In [54]:
from tensorflow.keras.models import Model

model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

In [55]:
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'binary_crossentropy'],
              metrics=[['accuracy'], ['accuracy']])

In [56]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 64)        │      1,280 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 32)        │      2,080 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 16)        │        528 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 16)        │        528 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 3)         │         51 │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 2)         │         34 │ dense_18[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,501 (17.58 KB)

 Trainable params: 4,501 (17.58 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.fit(X_train_scaled, [y_train_encoded_department, y_train_encoded_attrition], epochs=50, batch_size=32)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 572us/step - dense_17_accuracy: 0.3233 - dense_19_accuracy: 0.5058 - loss: 1.8247 
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - dense_17_accuracy: 0.6610 - dense_19_accuracy: 0.8421 - loss: 1.3400
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - dense_17_accuracy: 0.6486 - dense_19_accuracy: 0.8205 - loss: 1.2257
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - dense_17_accuracy: 0.6380 - dense_19_accuracy: 0.8279 - loss: 1.1920
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - dense_17_accuracy: 0.6364 - dense_19_accuracy: 0.8344 - loss: 1.1494
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - dense_17_accuracy: 0.6480 - dense_19_accuracy: 0.8305 - loss: 1.1236
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step - dense_17_accuracy: 0.6698 - dense_19_accuracy: 0.8368 - loss: 1.0911
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - dense_17_accuracy: 0.6832 - dense_19_accuracy: 0.8542 - loss: 1.0313
Epoch 9/50
37/3

In [59]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_encoded_department shape:", y_train_encoded_department.shape)
print("y_train_encoded_attrition shape:", y_train_encoded_attrition.shape)


X_train_scaled shape: (1176, 19)
y_train_encoded_department shape: (1176, 3)
y_train_encoded_attrition shape: (1176, 2)


In [60]:
model.fit(X_train_scaled, [y_train_encoded_department, y_train_encoded_attrition], epochs=50, batch_size=32)


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - dense_17_accuracy: 0.9299 - dense_19_accuracy: 0.9326 - loss: 0.4114
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - dense_17_accuracy: 0.9440 - dense_19_accuracy: 0.9465 - loss: 0.3651
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - dense_17_accuracy: 0.9493 - dense_19_accuracy: 0.9546 - loss: 0.3220
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - dense_17_accuracy: 0.9499 - dense_19_accuracy: 0.9409 - loss: 0.3445
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - dense_17_accuracy: 0.9445 - dense_19_accuracy: 0.9386 - loss: 0.3408
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - dense_17_accuracy: 0.9567 - dense_19_accuracy: 0.9455 - loss: 0.3211
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - dense_17_accuracy: 0.9643 - dense_19_accuracy: 0.9528 - loss: 0.3044
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - dense_17_accuracy: 0.9652 - dense_19_accuracy: 0.9621 - loss: 0.2727
Epoch 9/50
37/37

In [66]:
# Evaluate the model on the test dataset
evaluation_results = model.evaluate(X_test_scaled, [y_test_encoded_department, y_test_encoded_attrition], verbose=0)

# Print the evaluation results
print("Evaluation results:", evaluation_results)




Evaluation results: [3.9788553714752197, 0.5850340127944946, 0.7993197441101074]


In [69]:
# Print the department accuracy
print("Department Accuracy:", evaluation_results[1])

# Print the attrition accuracy
print("Attrition Accuracy:", evaluation_results[2])



Department Accuracy: 0.5850340127944946
Attrition Accuracy: 0.7993197441101074


In [71]:
questions = [
    "Is accuracy the best metric to use on this data? Why or why not?",
    "What activation functions did you choose for your output layers, and why?",
    "Can you name a few ways that this model could be improved?"
]

answers = [
    "1. Accuracy might not be the best metric for this data, especially if the classes are imbalanced or if different types of errors have varying costs. For example, in scenarios where false positives and false negatives have different consequences, metrics like precision, recall, or F1 score might be more informative.",
    "2. For the output layers, I would choose the softmax activation function for the multi-class classification task of predicting the department, as it normalizes the output into a probability distribution over the classes. For the binary classification task of predicting attrition, I would use the sigmoid activation function, as it squashes the output into the range [0, 1], representing the probability of attrition.",
    "3. Several ways to improve the model include:\n\
       - Increasing the size or complexity of the neural network architecture.\n\
       - Tuning hyperparameters such as learning rate, batch size, and number of epochs.\n\
       - Applying techniques like dropout regularization to prevent overfitting.\n\
       - Conducting feature engineering to extract more informative features.\n\
       - Addressing class imbalance through techniques like oversampling, undersampling, or using class weights.\n\
       - Exploring different optimization algorithms or learning rate schedules.\n\
       - Performing more thorough data preprocessing, including handling missing values and scaling features appropriately."
]

# Output questions and answers
for i, question in enumerate(questions):
    print("Question {}: {}".format(i+1, question))
    print("Answer:")
    print(answers[i])
    print()


Question 1: Is accuracy the best metric to use on this data? Why or why not?
Answer:
1. Accuracy might not be the best metric for this data, especially if the classes are imbalanced or if different types of errors have varying costs. For example, in scenarios where false positives and false negatives have different consequences, metrics like precision, recall, or F1 score might be more informative.

Question 2: What activation functions did you choose for your output layers, and why?
Answer:
2. For the output layers, I would choose the softmax activation function for the multi-class classification task of predicting the department, as it normalizes the output into a probability distribution over the classes. For the binary classification task of predicting attrition, I would use the sigmoid activation function, as it squashes the output into the range [0, 1], representing the probability of attrition.

Question 3: Can you name a few ways that this model could be improved?
Answer:
3. Se